In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

In [15]:
norm_params = pd.read_csv("norm_params.csv")
norm_params

,carat,depth,table,x,y,z,price,cut_Good,cut_Ideal,cut_Premium,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,5.01,79.0,95.0,10.74,58.9,31.8,9.842,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.20,43.0,44.0,0.00,0.0,0.0,5.787,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
%load_ext sql
%sql sqlite:///Diamonds

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [19]:
train_df = %sql select * from Diamond_reworked
train_df = train_df.DataFrame()
train_df = train_df.set_index("index")
train_df[:5]

 * sqlite:///Diamonds
Done.


,carat,depth,table,price,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
index,,,,,,,,,,,,,,,,,,,,,
0,0.020790,0.538889,0.274510,0.139581,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.168399,0.547222,0.235294,0.837485,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.108108,0.522222,0.294118,0.541554,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.182952,0.561111,0.254902,0.637238,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.033264,0.536111,0.294118,0.197534,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [21]:
Y_train = train_df["price"]
Y_train[:5]

/tmp/ipykernel_6074/598370826.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  Y_train[:5]


index
0    0.139581
1    0.837485
2    0.541554
3    0.637238
4    0.197534
Name: price, dtype: float64

In [23]:
X_train = train_df.drop(columns=("price"))
X_train[:5]

,carat,depth,table,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
index,,,,,,,,,,,,,,,,,,,,
0,0.020790,0.538889,0.274510,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.168399,0.547222,0.235294,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.108108,0.522222,0.294118,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.182952,0.561111,0.254902,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.033264,0.536111,0.294118,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [37]:
X_test = pd.read_csv("/home/user/projects_jupyter/Diamond/diamonds-databcn0722/test.csv")
X_test = X_test.set_index("id")
X_test[:5]

,carat,cut,color,clarity,depth,table,x,y,z
id,,,,,,,,,
0,0.32,Ideal,I,SI1,60.5,58.0,4.43,4.49,2.70
1,1.24,Premium,I,SI1,62.9,60.0,6.80,6.74,4.26
2,1.66,Premium,D,SI1,62.0,59.0,7.55,7.60,4.70
3,0.75,Premium,D,SI2,60.6,56.0,5.94,5.90,3.59
4,1.50,Fair,E,SI2,64.8,55.0,7.26,7.15,4.67


In [38]:
Y_test = pd.read_csv("/home/user/projects_jupyter/Diamond/diamonds-databcn0722/sample_submission.csv")
Y_test = Y_test.set_index("id")
Y_test[:5]

,price
id,
0,7.547901
1,7.049216
2,8.696350
3,8.515766
4,7.334336
